# Lab 4: SOLID Principles in Python
**Author:** Teoman Kaman
**Date:** 21 Oct

This notebook demonstrates the five SOLID principles with both good and bad examples for each principle.

## S - Single Responsibility Principle (SRP)

**Explanation:** SRP is class need to be atomic in the context of one class needs to serve one purpose; one responsibility per class. 

**Why it matters:** It is important because if one class has multiple responsibility then it is harder to maintain. As well as when class is SRP it is easier to write unit tests. 

### Bad Example - Violates SRP

In [9]:
class UserManagerBad:
    """This class violates SRP by handling multiple responsibilities."""
    
    def __init__(self):
        self.users = {}          # persistence responsibility
        self.audit_log = []      # logging responsibility

    def add_user(self, username, password, email):
        if len(password) < 8:                 # validation/auth responsibility
            raise ValueError("Weak password")
        self.users[username] = (password, email)
        self.log_activity(f"User created: {username}")
        self.send_email(email)

    def log_activity(self, message):
        self.audit_log.append(message)        # logging/auditing responsibility

    def send_email(self, email):
        print(f"Sending promo email to {email}")  # notification responsibility
        
        
print("UserManagerBad Example:")
bad_manager = UserManagerBad()

# add_user exercises validation, persistence, logging, and triggers email send
bad_manager.add_user("john_doe", "123456789", "johndoe@gmail.com")

# confirm the “database” and audit log side effects
print(bad_manager.users)
print(bad_manager.audit_log)

# call auxiliary methods directly to show they’re baked into the same class
bad_manager.log_activity("Manual audit entry")
bad_manager.send_email("johndoe@gmail.com")

UserManagerBad Example:
Sending promo email to johndoe@gmail.com
{'john_doe': ('123456789', 'johndoe@gmail.com')}
['User created: john_doe']
Sending promo email to johndoe@gmail.com


**What's wrong with this approach:**
- In one class there is adding user logging activity and sending email those three actions are unrelated thus this class has 3 responsibility instead of 1
-  responsibilites are password checking, audit logging, email output
- When a email format changes or another part; because of changes it can break other features as well. Thus it makes it harder to maintain because the other functions could have dependency. 

### Good Example - Follows SRP

In [11]:
class UserManagerGood:
    """Handles user lifecycle only; delegates emails elsewhere."""
    def __init__(self, email_service):
        self.email_service = email_service
        self.users = {}

    def add_user(self, username, password, email):
        if len(password) < 8:
            raise ValueError("Weak password")
        self.users[username] = {"password": password, "email": email}
        self.email_service.send_email(email)

    def get_user(self, username):
        return self.users.get(username)


class EmailService:
    """Owns all outbound email behavior."""
    def send_email(self, email):
        print(f"Sending welcome email to {email}")


email_service = EmailService()
good_manager = UserManagerGood(email_service)

good_manager.add_user("jane_doe", "securepass", "jane@example.com")
print(good_manager.get_user("jane_doe"))
# {'password': 'securepass', 'email': 'jane@example.com'}

good_manager.add_user("adam_smith", "anotherpw", "adam@example.com")


Sending welcome email to jane@example.com
{'password': 'securepass', 'email': 'jane@example.com'}
Sending welcome email to adam@example.com


**Why this is better:**
- The two classes we have now is atomic meaning that they have only one responsiblity
-  UserManagerGood handles the adding and getting user so its just user rules. While EmailService class only takes care of sending email to the user
- Any rule change in classes won't affect each other because we will change the class with that responsiblity thus it makes less risk of breaking other features. 

## O - Open/Closed Principle (OCP)

**Explanation:** It means that classes, functions, modules should be open for extension but closed for modification. 

**Why it matters:** It matters because when the code is live we dont want to keep editing classes every time new feature needs to be implemented extending instead of modfying reduces bugs as well. Also OCP lets mutliple features evlopve in parallel without depending on each other

### Bad Example - Violates OCP

In [14]:
class UserManagerBad:
    """Violates OCP: every new notification mode requires editing this method."""
    
    def __init__(self):
        self.users = {}

    def add_user(self, username, email, notification):
        self.users[username] = email
        if notification == "email":
            print(f"Welcome email to {email}")
        elif notification == "sms":
            print(f"Welcome SMS to {username}")
        # Adding push, in-app, etc. means touching this method again.

print("UserManagerBad Example:")
bad_manager = UserManagerBad()
bad_manager.add_user("john_doe", "john@example.com", "email")
bad_manager.add_user("jane_doe", "jane@example.com", "sms")
print(bad_manager.users)


UserManagerBad Example:
Welcome email to john@example.com
Welcome SMS to jane_doe
{'john_doe': 'john@example.com', 'jane_doe': 'jane@example.com'}


**What's wrong with this approach:**
- It is wrong because add_user function keeps growing as new notification type needs be done
- add_user function both stores teh users and decides how each notification type should behave
- It becomes hard to maintain because each new feature forces us to revisit and edit the same method again, increasing the chance of mistakes and making the logic more tangled over time.

### Good Example - Follows OCP

In [15]:
class UserManagerGood:
    """Open for new notification behaviors, closed for code changes."""
    
    def __init__(self, notifier):
        self.users = {}
        self.notifier = notifier   # plug in any notifier you need

    def add_user(self, username, email):
        self.users[username] = email
        self.notifier.notify(username, email)


class EmailNotifier:
    def notify(self, username, email):
        print(f"Welcome email → {email}")


class SmsNotifier:
    def notify(self, username, email):
        print(f"Welcome SMS → {username}")


print("UserManagerGood Example:")
email_manager = UserManagerGood(EmailNotifier())
email_manager.add_user("alice", "alice@example.com")

sms_manager = UserManagerGood(SmsNotifier())
sms_manager.add_user("bob", "bob@example.com")

print(email_manager.users)
print(sms_manager.users)


UserManagerGood Example:
Welcome email → alice@example.com
Welcome SMS → bob
{'alice': 'alice@example.com'}
{'bob': 'bob@example.com'}


**Why this is better:**
- In this way the class keeps growing in new notifying methods so we extend the class but we dont modify the existing function. This lowers the risk of breaking what already was working
- The manager owns user registration while each notifier class represent each notifying type. That clear division keeps the logic focused and easier to understand.
- When new notification type arrive as new class the other classes stay stable. When requirements change we just swap or update the corresponding notifier class without rewriting user management codes


## L - Liskov Substitution Principle (LSP)

**Explanation:** A subclass should be able replace its parents class without changing the correctness so it inherits the attributes of the parent class without changing the correctness

**Why it matters:** This ensures the reliability in inheritence so that nothing suprises us when class inherits parent class. It makes the code predictable and makes it easier to find bugs

### Bad Example - Violates LSP

In [17]:
class UserManagerBad:
    """Violates LSP: Subclass changes base class behavior unexpectedly."""
    
    def add_user(self, username):
        print(f"User {username} created")
        return True  # callers expect a success flag


class ReadOnlyUserManager(UserManagerBad):
    """Supposed to behave like UserManagerBad, but breaks the contract."""
    
    def add_user(self, username):
        # Abruptly rejects the operation, so code depending on True now crashes.
        raise PermissionError("Read-only manager cannot add users")


print("LSP Violation Demo:")
manager = UserManagerBad()
readonly_manager = ReadOnlyUserManager()

def onboard(user_manager, username):
    # Works with UserManagerBad, but explodes with ReadOnlyUserManager.
    if user_manager.add_user(username):
        print("Welcome flow triggered.")

onboard(manager, "alice")
onboard(readonly_manager, "bob")


LSP Violation Demo:
User alice created
Welcome flow triggered.


PermissionError: Read-only manager cannot add users

**What's wrong with this approach:**
- ReadOnlyUserManager stops inheriting the base class promise that add_user returns a success flag, so any caller that expects the original behavior crashes. Because the subclass alters the contract instead of extending it you can’t safely substitute it for the base type.
- The base class is responsible for creating users, while the subclass quietly adds access-control enforcement inside the same method. Those mixed responsibilities cause the subclass to block core behavior rather than extend it.
- Every caller now needs custom guards or exception handling when a different subclass is injected, complicating the codebase. Future changes to onboarding must worry about which version of add_user they’re calling and how it might misbehave.

### Good Example - Follows LSP


In [20]:
class UserManagerGood:
    """Base behavior: create user and signal success."""
    
    def add_user(self, username):
        print(f"User {username} created")
        return True


class AuditingUserManager(UserManagerGood):
    """Extends base behavior without changing expectations."""
    
    def __init__(self):
        self.audit_log = []

    def add_user(self, username):
        result = super().add_user(username)  # still returns True
        if result:
            self.audit_log.append(f"created {username}")
        return result


print("Good example of LSP:")
manager = UserManagerGood()
audited_manager = AuditingUserManager()

def onboard(user_manager, username):
    if user_manager.add_user(username):
        print("Welcome flow triggered.")

onboard(manager, "alice")
onboard(audited_manager, "bob")
print(audited_manager.audit_log)


Good example of LSP:
User alice created
Welcome flow triggered.
User bob created
Welcome flow triggered.
['created bob']


**Why this is better:**
- This way the subclass correctly inherits the parent class and original behaviour thus existing features works same and we can reuse the existing code without surprises
- UserManagerGood creates users, while AuditingUserManager adds tracking. Each class does its own job without stepping on the other.
- Because logging lives in the auditing subclass and user creation stays in the base class, each change touches only one place. That makes updates simpler, lowers the risk of bugs, and lets developers add new subclasses without rewriting existing code.


## I - Interface Segregation Principle (ISP)

**Explanation:** ISP is a class should not contain methods its not going to use. We should implement functions that class will use and what actually needs

**Why it matters:** It is important because it makes the class cleaner and more organized. Again it is easier to maintain and understand what class is doing. 

### Bad Example - Violates ISP


In [ ]:
class UserNotifier:
    """Too many responsibilities baked into one interface."""
    def send_email(self, address, message):
        raise NotImplementedError

    def send_sms(self, phone, message):
        raise NotImplementedError

    def send_push(self, username, message):
        raise NotImplementedError


class EmailOnlyNotifier(UserNotifier):
    """Needs only email, but must stub everything else."""
    def send_email(self, address, message):
        print(f"Email to {address}: {message}")

    def send_sms(self, phone, message):
        # Forced to supply meaningless implementation
        raise NotImplementedError("This notifier cannot send SMS")

    def send_push(self, username, message):
        raise NotImplementedError("This notifier cannot send push notifications")


notifier = EmailOnlyNotifier()
notifier.send_email("user@example.com", "Hi!")
notifier.send_sms("123-456-7890", "Hello!")  # blows up



Email to user@example.com: Hi!
